# Introdução

Esse notebook tem objetivo de entender os princípios de estimações de pontuação de crédito bancário.  O crédito faz parte da vida de muitas pessoas mas para fornecer esse tipo de serviço é necessário entender os riscos envolvidos em emprestimos pessoais e consignados, cartões de credito, financiamento de imóveis e veículos, apesar de todos esses exemplo, pouco se fala sobre como funciona a tomada de decisão por trás do sistema de crédito. 

> [Credit Scoring Google Scholar](https://scholar.google.com.br/scholar?hl=pt-PT&as_sdt=0%2C5&as_vis=1&q=credit+scoring&btnG=)


## Cadeia de crédito

<img src='https://raw.githubusercontent.com/ConradBitt/Credit_Scoring/main/images/ciclo_credito.png'  width=50%>

Os pilares da cadeia de crédito consiste em ciclo de quatro etapas que fundamentam o funcionamento do sistema, eles são 

* Prospecção 

* Concessão 

* Gestão de Risco 

* Recuperação 


Neste notebook vou focar em quais métricas de avaliação são  utilizadas na etapa de **concessão**. 

Um ponto importante na cadeia de crédito são os grupos de clientes, eles se dividem em dois **poupadores** e **tomadores**. Os poupadores são clientes que reservam suas economias para extrair rendimentos no futuro e os tomadores são clientes que tomam emprestado uma quantia hoje para pagar no futuro. Quem faz a intermediação entre popuadores e tomadores são as instituições financeiras. (*Lembrando que apenas instituições podem fazer essa mediação, pessoas físicas emprestando valores monetarios a troco de juros pode caracterizar o crime de agiotagem.*)

## Pedido de crétido 

<img src='https://raw.githubusercontent.com/ConradBitt/Credit_Scoring/main/images/pedido_credito.png'>

O pedido de credito se inicia com o **tomador** indo até uma **instituição financeira** e ela através de um **modelo de decisão** e este modelo irá dizer se o **tomador** terá um credito **aprovado** ou **rerejeitado**. 

## Credit Scoring

O Credit Scoring consiste em analisar a probabilidade (0 ou 1) de um evento ocorrer dado um conjunto de variáveis aleatórias a respeito dos fenômenos envolvidos. Agora pensando na concessão o tipo de evento que se busca estimar é se um tomador de crédito será inadimplente ou não, ou seja, **é um problema de classificação binária**.

Basicamente toda análise de concessão de crédito consiste em analisar o risco de uma operação, por parte da instituição financeira, o risco é medido **como a expectativa de receber um montante em um determinado período de tempo não ocorrer**. Esse risco é incluido de forma parcelada através dos juros cobrados ao tomador do credito..



# Importações de módulos python

In [2]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib as mpl


from matplotlib import pyplot as plt

In [3]:
print('~~ Versão Python ~~')
!python --version

print('\n\n~~ Versões Módulos ~~')

print(f'numpy: {np.__version__}')
print(f'pandas: {pd.__version__}')
print(f'seaborn: {sns.__version__}')
print(f'matplotlib: {mpl.__version__}')

~~ Versão Python ~~
Python 3.7.9


~~ Versões Módulos ~~
numpy: 1.20.1
pandas: 1.2.4
seaborn: 0.11.1
matplotlib: 3.3.4


# Base de dados

Os dados que serão utilizados neste trabalho são da UCI Machine Learning Repository, mais precisamente o German Credit Data, disponível em: 

> https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29

In [4]:
credit_score = pd.read_csv('./dados/german_credit.csv', header=0)

In [6]:
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   default                     1000 non-null   int64 
 1   account_check_status        1000 non-null   object
 2   duration_in_month           1000 non-null   int64 
 3   credit_history              1000 non-null   object
 4   purpose                     1000 non-null   object
 5   credit_amount               1000 non-null   int64 
 6   savings                     1000 non-null   object
 7   present_emp_since           1000 non-null   object
 8   installment_as_income_perc  1000 non-null   int64 
 9   personal_status_sex         1000 non-null   object
 10  other_debtors               1000 non-null   object
 11  present_res_since           1000 non-null   int64 
 12  property                    1000 non-null   object
 13  age                         1000 non-null   int64

> Através das informações desse dataset podemos verificar que são 100 observações dividida oito variáveis do tipo `int` e treze variáveis do tipo `object`.